# Eat Safe, Love

## Notebook Set Up

In [ ]:
from pymongo import MongoClient
import pandas as pd
from pprint import pprint

In [ ]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [ ]:
# assign the uk_food database to a variable name
db = mongo['uk_food']

In [ ]:
# review the collections in our database
print(db.list_collection_names())

In [ ]:
# assign the collection to a variable
establishments = db['establishments']

## Part 3: Exploratory Analysis
Unless otherwise stated, for each question: 
* Use `count_documents` to display the number of documents contained in the result.
* Display the first document in the results using `pprint`.
* Convert the result to a Pandas DataFrame, print the number of rows in the DataFrame, and display the first 10 rows.

### 1. Which establishments have a hygiene score equal to 20?

In [ ]:
# Find the establishments with a hygiene score of 20
#'scores': {'ConfidenceInManagement': 5, 'Hygiene': 5, 'Structural': 5}}

# Use count_documents to display the number of documents in the result
query = {'scores.Hygiene': 20}
results01 = establishments.count_documents(query)
print(f"{results01} establishments have a hygiene score equal to 20.")
    
# Display the first document in the results using pprint
results02 = establishments.find(query).limit(1)
pprint(list(results02))

In [ ]:
# Convert the result to a Pandas DataFrame
from pymongo import MongoClient
import pandas as pd
from pprint import pprint

query = {'scores.Hygiene': 20}
results02 = establishments.find(query).limit(1)

# a. Convert results to list
# initialize empty list
document_list = []
# iterate over cursor object
for document in results02:
    document_list.append(document)
# document_list now contains all documents from results02
pprint(document_list)

#b. Create Pandas DataFrame from list
result_df = pd.DataFrame()
result_df = pd.DataFrame(document_list)
#print(result_df)

# Display the number of rows in the DataFrame
print("Number of rows in result_df:", result_df.shape[0])

# Display the first 10 rows of the DataFrame
print(result_df.head())

### 2. Which establishments in London have a `RatingValue` greater than or equal to 4?

In [ ]:
# Find the establishments with London as the Local Authority and has a RatingValue greater than or equal to 4.
# 'City of London Corporation'

query = {'$and': [
        {'LocalAuthorityName': {'$regex': "London"}},
        {'RatingValue': {'$gte': 4}}
        ]}

results1 = establishments.find(query)

# Use count_documents to display the number of documents in the result
results2 = establishments.count_documents(query)
print(f"{results2} establishments in London have a rating value greater than or equal to 4.")

# Display the first document in the results using pprint
results3 = establishments.find(query).limit(1)
pprint(list(results3))

In [ ]:
# Convert the result to a Pandas DataFrame
results3 = establishments.find(query).limit(1)
# a. Convert results to list
document_list = []
for document in results3:
    document_list.append(document)
# document_list now contains all documents from results3
pprint(document_list)

#b. Create Pandas DataFrame from list
result_df = pd.DataFrame()
result_df = pd.DataFrame(document_list)

# Display the number of rows in the DataFrame
print("Number of rows in result_df:", result_df.shape[0])

# Display the first 10 rows of the DataFrame
print(result_df.head())

### 3. What are the top 5 establishments with a `RatingValue` rating value of 5, sorted by lowest hygiene score, nearest to the new restaurant added, "Penang Flavours"?

In [ ]:
# Search within 0.01 degree on either side of the latitude and longitude.
# Rating value must equal 5
# Sort by hygiene score

#degree_search = 0.01   #latitude =   #longitude =   #query =   #sort =    #limit = 5

# https://bootcampspot.instructure.com/courses/4981/external_tools/ provided assistance

#To iterate over documents in collection, must first use find() method to retrieve cursor object
#establishments.find({}) retrieves all documents in 'establishments' collection and returns cursor object that can be
#iterated over to access each document
cursor = establishments.find({})  # Retrieve all documents in collection

# Define establishment name to search for
establishment_name = 'Penang Flavours'

# Define search radius (0.01 degree)
degree_search = 0.01

# Define empty list to store RativgValue5 establishments within specified range
rated_establishments_within_radius = []

# Iterate over documents to find establishment "at center" based on BusinessName and retrieve its lat, lon values   
for doc in cursor:
    if doc.get('BusinessName') == establishment_name:
        center_latitude = doc.get('geocode').get('latitude')
        center_longitude = doc.get('geocode').get('longitude')

        # Define latitude and longitude ranges based on center latitude, center longitude, and search radius
        latitude_range = [center_latitude - degree_search, center_latitude + degree_search]
        longitude_range = [center_longitude - degree_search, center_longitude + degree_search]

        # Construct query to find establishments within specified range
        query = {
            "geocode.latitude": {"$gte": latitude_range[0], "$lte": latitude_range[1]},
            "geocode.longitude": {"$gte": longitude_range[0], "$lte": longitude_range[1]},
            "RatingValue": 5
        }       
    
#Get query results in form of list, because .find must be used on lists
rated_establishments_within_radius = list(establishments.find(query))

# Count number of documents in query result using count_documents
num_documents = establishments.count_documents(query)

# Print the number of documents in query result
print("Number of documents in the query result:", num_documents)
           
# Display first document in list
pprint(rated_establishments_within_radius[:1])

In [ ]:
# Sort by hygiene score
#Assistance provided by https://bootcampspot.instructure.com/courses/4981/external_tools/313

# Assuming establishments is collection and Hygiene is nested within scores
# Sort list by hygiene value in ascending order
sorted_establishments = sorted(rated_establishments_within_radius, key=lambda x: x['scores']['Hygiene'])

# Display one document in sorted_establishments list
pprint(sorted_establishments[:1])

# Convert sorted_establishments list of dictionaries to Pandas DataFrame
sorted_establishments_df = pd.DataFrame()
sorted_establishments_df = pd.DataFrame(sorted_establishments)

num_rows = sorted_establishments_df.shape[0]
print("Number of rows in the DataFrame:", num_rows)

In [ ]:
# Print first 10 rows of data frame
pprint(sorted_establishments_df.head(10))

### 4. How many establishments in each Local Authority area have a hygiene score of 0?

In [ ]:
# Create a pipeline that:
# 1. Matches establishments with a hygiene score of 0
# 2. Groups the matches by Local Authority
# 3. Sorts the matches from highest to lowest

# https://bootcampspot.instructure.com/courses/4981/external_tools/ provided assistance
# Write match query to find only documents with hygiene score of 0
match_query = {'$match': {'scores.Hygiene': 0}}

#Output of previous query serves as input for this next query in aggregation pipeline
group_query = {'$group': {'_id': "$LocalAuthorityName", 'count': { '$sum': 1 }}}

# Create dictionary that will allow pipeline to sort by LocalAuthorityName count in descending order
sort_values = {'$sort': { 'count': -1}}

# Put pipeline together
pipeline = [match_query, group_query, sort_values]              
result = establishments.aggregate(pipeline)  

# Iterate over the cursor to get the actual documents
documents = list(result)

# Count the number of documents in the query result
num_documents = len(documents)

# Print the number of documents in the query result
print("Number of documents in the query result:", num_documents)

# Display the first document in the list
if documents:     #checks if documents list is not empty; if documents: will be True if documents list contains at least one document
    pprint(documents[0])   #print the first doc (index 0) in documents

#Convert the result to a Pandas DataFrame, print the number of rows in the DataFrame, and display the first 10 rows.
documents_df = pd.DataFrame()
documents_df = pd.DataFrame(documents)

num_rows = documents_df.shape[0]
print("Number of rows in the DataFrame:", num_rows)

# Display the first 10 rows of the DataFrame
print(documents_df.head(10))
